-----------------------------------------------------
# Tech Challenge (Fase 1): ⚙️ Setup e Pré-processamento - Análise de Câncer de Pele
-----------------------------------------------------
Aluno: [Fernando]

---
**❗ ATENÇÃO**

Este notebook automatiza a preparação do ambiente.

**O que ele faz:**
- Baixa o dataset HAM10000 e as imagens do Kaggle na pasta `/data`.
- Instala as bibliotecas necessárias.
- Cria os scripts de pré-processamento de dados na pasta `/src`.
- Cria também os scripts para uso da API e do Dashboard (frontend) na pasta `/app`.

**Execute todo o conteúdo deste notebook primeiro para preparar o projeto para os próximos notebooks: 02_Analise_e_Modelagem e 03_Visao_Computacional_CNN.**

Após baixar todo o conteúdo do reposítório [https://github.com/Ferstuque/skin_cancer_analysis](https://github.com/Ferstuque/skin_cancer_analysis), ao rodar este notebook ele irá extrair os datases (metadatos e imagens) e criará os *scripts* automaticamente juntamente com suas respectivas pastas.

**Obs.: mantenha o caminho da pasta de projeto `PROJECT_PATH` o mesmo em todos os Notebooks.**

## Download, Extração, Limpeza e Preparação dos dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/Tech Challenge/skin_cancer_analysis'
os.makedirs(PROJECT_PATH, exist_ok=True)
os.chdir(PROJECT_PATH)
print(f"Diretório atual: {os.getcwd()}")

Diretório atual: /content/drive/MyDrive/Colab Notebooks/Tech Challenge/skin_cancer_analysis


In [ ]:
# Instalando dependências
!pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# será solicitado o arquivo 'kaggle.json' nesta etapa
print("Faça o upload do seu arquivo kaggle.json")
files.upload()

# Mova o arquivo para o local correto e configure as permissões (o dataset será baixado na pasta 'data')
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 -p ./data --unzip

Faça o upload do seu arquivo kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


## Script de leitura e pré-processamento do dataset

In [ ]:
import os

# Garantir que o diretório 'src' exista na sua pasta de projeto no Drive
# ❗ Isso só precisa ser executado uma vez.
os.makedirs('src', exist_ok=True)
print("Diretório 'src' criado.")

Diretório 'src' criado.


In [ ]:
%%writefile src/data_preprocessing.py

import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

def preprocess_tabular_data(csv_path):
    """
    Carrega e pré-processa os dados tabulares do dataset HAM10000.

    Esta função realiza as seguintes etapas:
    1. Carrega os dados do arquivo CSV.
    2. Preenche os valores ausentes na coluna 'age' com a mediana.
    3. Aplica One-Hot Encoding nas variáveis categóricas ('dx_type', 'sex', 'localization').
    4. Aplica StandardScaler na variável numérica 'age'.
    5. Limpa e padroniza TODOS os nomes de colunas como etapa final.
    """
    print(f"Carregando dados de: {csv_path}")
    df = pd.read_csv(csv_path)

    # --- 1. Tratamento de Valores Ausentes ---
    median_age = df['age'].median()
    df['age'].fillna(median_age, inplace=True)
    print(f"Valores nulos em 'age' preenchidos com a mediana ({median_age}).")

    # --- 2. Codificação de Variáveis Categóricas ---
    categorical_cols = ['dx_type', 'sex', 'localization']
    print(f"Aplicando One-Hot Encoding em: {categorical_cols}")
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_data = encoder.fit_transform(df[categorical_cols])
    encoded_cols_names = encoder.get_feature_names_out(categorical_cols)
    encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols_names, index=df.index)
    df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

    # --- 3. Escalonamento de Variáveis Numéricas ---
    print("Aplicando StandardScaler na coluna 'age'.")
    scaler = StandardScaler()
    df['age'] = scaler.fit_transform(df[['age']])

    # --- 4. Limpeza Final dos Nomes das Colunas ---
    print("Limpando os nomes das colunas...")
    df.columns = df.columns.str.replace(' ', '_', regex=False)
    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
    print("Nomes das colunas padronizados.")

    print("\nPré-processamento concluído com sucesso!")

    return df

Overwriting src/data_preprocessing.py


## Testando o script de pré-processamento

In [ ]:
# Importar o módulo que acabamos de criar acima, usando um alias 'dp' para facilitar
import src.data_preprocessing as dp

# Definir o caminho para o arquivo de metadados
METADATA_PATH = './data/HAM10000_metadata.csv'

# Chamar a função de pré-processamento
# A função irá carregar, limpar, transformar os dados e retornar o resultado
processed_df = dp.preprocess_tabular_data(METADATA_PATH)

print("\n--- Visualização do DataFrame Pré-processado ---")

display(processed_df.head())

print(f"\nShape do DataFrame processado: {processed_df.shape}")
print("\nTipos de dados das colunas:")
print(processed_df.info())

Carregando dados de: ./data/HAM10000_metadata.csv
Valores nulos em 'age' preenchidos com a mediana (50.0).
Aplicando One-Hot Encoding em: ['dx_type', 'sex', 'localization']
Aplicando StandardScaler na coluna 'age'.
Limpando os nomes das colunas...
Nomes das colunas padronizados.

Pré-processamento concluído com sucesso!

--- Visualização do DataFrame Pré-processado ---


/content/drive/MyDrive/Colab Notebooks/Tech Challenge/skin_cancer_analysis/src/data_preprocessing.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(median_age, inplace=True)


,lesion_id,image_id,dx,age,dx_type_confocal,dx_type_consensus,dx_type_follow_up,dx_type_histo,sex_female,sex_male,...,localization_face,localization_foot,localization_genital,localization_hand,localization_lower_extremity,localization_neck,localization_scalp,localization_trunk,localization_unknown,localization_upper_extremity
0,HAM_0000118,ISIC_0027419,bkl,1.663522,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,HAM_0000118,ISIC_0025030,bkl,1.663522,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,HAM_0002730,ISIC_0026769,bkl,1.663522,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,HAM_0002730,ISIC_0025661,bkl,1.663522,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,HAM_0001466,ISIC_0031633,bkl,1.368014,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Shape do DataFrame processado: (10015, 26)

Tipos de dados das colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   lesion_id                     10015 non-null  object 
 1   image_id                      10015 non-null  object 
 2   dx                            10015 non-null  object 
 3   age                           10015 non-null  float64
 4   dx_type_confocal              10015 non-null  float64
 5   dx_type_consensus             10015 non-null  float64
 6   dx_type_follow_up             10015 non-null  float64
 7   dx_type_histo                 10015 non-null  float64
 8   sex_female                    10015 non-null  float64
 9   sex_male                      10015 non-null  float64
 10  sex_unknown                   10015 non-null  float64
 11  localization_abdomen          10015 non-null  

## Script da API

In [ ]:
# Garantir que o diretório 'app' exista na sua pasta de projeto no Drive
# ❗ Isso só precisa ser executado uma vez.
os.makedirs('app', exist_ok=True)
print("Diretório 'app' criado.")

Diretório 'app' criado.


In [ ]:
%%writefile app/main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import joblib
import pandas as pd
import os

# --- 1. Configuração da Aplicação ---
app = FastAPI(
    title="API de Diagnóstico de Câncer de Pele",
    version="1.0.0",
    description="Uma API para prever a probabilidade de uma lesão de pele ser Melanoma usando o melhor modelo XGBoost treinado."
)

# --- 2. Carregamento do Modelo e Colunas ---
# Caminhos relativos à raiz do projeto onde o servidor uvicorn será executado
model_path = "saved_models/best_model_v1.pkl"
columns_path = "saved_models/model_columns.pkl"

try:
    model = joblib.load(model_path)
    model_columns = joblib.load(columns_path)
    print("Modelo e colunas carregados com sucesso.")
except FileNotFoundError:
    print(f"Erro: Arquivo do modelo ou colunas não encontrado. Verifique os caminhos: {model_path}, {columns_path}")
    model = None
    model_columns = None

# --- 3. Definição do Modelo de Entrada (Payload) ---
# Usamos Field para definir valores padrão, tornando a API mais fácil de testar
class SkinLesionFeatures(BaseModel):
    age: float = Field(..., example=55.0, description="Idade do paciente")
    dx_type_confocal: int = Field(0, example=0, description="Diagnóstico por microscopia confocal")
    dx_type_consensus: int = Field(0, example=0, description="Diagnóstico por consenso de especialistas")
    dx_type_follow_up: int = Field(0, example=1, description="Lesão em acompanhamento")
    dx_type_histo: int = Field(0, example=0, description="Diagnóstico por histopatologia (biópsia)")
    sex_female: int = Field(0, example=0, description="Sexo feminino")
    sex_male: int = Field(1, example=1, description="Sexo masculino")
    sex_unknown: int = Field(0, example=0, description="Sexo desconhecido")
    localization_abdomen: int = Field(0, example=0)
    localization_acral: int = Field(0, example=0)
    localization_back: int = Field(0, example=0)
    localization_chest: int = Field(0, example=0)
    localization_ear: int = Field(0, example=0)
    localization_face: int = Field(0, example=0)
    localization_foot: int = Field(0, example=0)
    localization_genital: int = Field(0, example=0)
    localization_hand: int = Field(0, example=0)
    localization_lower_extremity: int = Field(1, example=1)
    localization_neck: int = Field(0, example=0)
    localization_scalp: int = Field(0, example=0)
    localization_trunk: int = Field(0, example=0)
    localization_unknown: int = Field(0, example=0)
    localization_upper_extremity: int = Field(0, example=0)

# --- 4. Endpoints da API ---
@app.get("/", tags=["Root"])
def read_root():
    """Endpoint raiz para verificar se a API está funcionando."""
    return {"message": "Bem-vindo à API de Diagnóstico. Use o endpoint /predict para fazer uma previsão."}

@app.post("/predict", tags=["Prediction"])
def predict(lesion_features: SkinLesionFeatures):
    """
    Recebe os dados de uma lesão de pele e retorna a previsão de Melanoma.
    """
    if not model or not model_columns:
        raise HTTPException(status_code=503, detail="Modelo não está disponível. Verifique os logs do servidor.")

    try:
        # Converter os dados de entrada para um DataFrame do Pandas
        data = pd.DataFrame([lesion_features.dict()])

        # Garantir que a ordem das colunas seja exatamente a mesma do treinamento
        data = data[model_columns]

        # Fazer a predição
        prediction_proba = model.predict_proba(data)[0]
        prediction = model.predict(data)[0]

        # Formatar a resposta
        probability_melanoma = prediction_proba[1]
        diagnosis = "Melanoma" if prediction == 1 else "Não-Melanoma"

        return {
            "diagnostico": diagnosis,
            "probabilidade_melanoma": float(probability_melanoma)
        }
    except Exception as e:
        # Captura qualquer outro erro durante a predição
        raise HTTPException(status_code=500, detail=f"Erro durante a predição: {str(e)}")

Writing app/main.py


## Script do Dashboard

In [ ]:
%%writefile app/dashboard.py
import streamlit as st
import pandas as pd
import requests
import json
import plotly.express as px

# --- 1. Configuração da Página ---
st.set_page_config(
    page_title="Skin-Cancer-Analysis",
    page_icon="🩺",
    layout="wide"
)

# --- 2. Título e Descrição ---
st.title("Sistema de Apoio ao Diagnóstico de Câncer de Pele 🩺")
st.markdown("""
Esta é uma interface de demonstração para um sistema de Machine Learning treinado para prever a probabilidade de uma lesão de pele ser um Melanoma.
O sistema utiliza um modelo **XGBoost** treinado com dados do dataset **HAM10000**.

**Atenção:** Este é um protótipo para fins educacionais e **não substitui o diagnóstico médico**. Consulte sempre um especialista.
""")

# --- 3. Layout do Formulário de Predição ---
st.header("Simulador de Diagnóstico")
st.markdown("Preencha os campos abaixo com as informações da lesão para receber uma predição do modelo.")

with st.form("prediction_form"):
    # Criando colunas para um layout mais organizado
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Informações do Paciente")
        age = st.slider("Idade do Paciente", min_value=0, max_value=100, value=50, step=1)
        sex = st.selectbox("Sexo", ["Masculino", "Feminino", "Desconhecido"], index=0)

    with col2:
        st.subheader("Características da Lesão")
        localization_options = [
            'abdômen', 'acrál', 'costas', 'tórax', 'orelha', 'rosto', 'pé',
            'genital', 'mão', 'extremidade inferior', 'pescoço', 'couro cabeludo',
            'tronco', 'desconhecida', 'extremidade superior'
        ]
        localization_map = {
            'abdômen': 'abdomen', 'acrál': 'acral', 'costas': 'back', 'tórax': 'chest',
            'orelha': 'ear', 'rosto': 'face', 'pé': 'foot', 'genital': 'genital',
            'mão': 'hand', 'extremidade inferior': 'lower extremity', 'pescoço': 'neck',
            'couro cabeludo': 'scalp', 'tronco': 'trunk', 'desconhecida': 'unknown',
            'extremidade superior': 'upper extremity'
        }
        localization_display = st.selectbox("Localização da Lesão", options=localization_options, index=9)
        localization_internal = localization_map[localization_display]

        dx_type = st.selectbox("Método de Confirmação Inicial", ["Histopatologia", "Acompanhamento", "Consenso", "Confocal"], index=1)

    # Botão de submissão do formulário
    submit_button = st.form_submit_button(label="Realizar Previsão")

# --- 4. Lógica de Predição e Exibição de Resultados ---
if submit_button:
    # --- Preparar o Payload para a API ---
    # Inicializar todas as features com 0
    features = {
        'age': age, 'dx_type_confocal': 0, 'dx_type_consensus': 0, 'dx_type_follow_up': 0,
        'dx_type_histo': 0, 'sex_female': 0, 'sex_male': 0, 'sex_unknown': 0,
        'localization_abdomen': 0, 'localization_acral': 0, 'localization_back': 0,
        'localization_chest': 0, 'localization_ear': 0, 'localization_face': 0,
        'localization_foot': 0, 'localization_genital': 0, 'localization_hand': 0,
        'localization_lower_extremity': 0, 'localization_neck': 0,
        'localization_scalp': 0, 'localization_trunk': 0, 'localization_unknown': 0,
        'localization_upper_extremity': 0
    }

    # Atualizar as features com base na seleção do usuário
    # Sexo
    if sex == 'Masculino': features['sex_male'] = 1
    elif sex == 'Feminino': features['sex_female'] = 1
    else: features['sex_unknown'] = 1

    # Tipo de diagnóstico
    dx_type_key = f"dx_type_{dx_type.lower()}"
    if dx_type_key in features:
        features[dx_type_key] = 1

    # Localização
    localization_key = f"localization_{localization_internal.replace(' ', '_')}"
    if localization_key in features:
        features[localization_key] = 1

    # --- Chamar a API FastAPI ---
    with st.spinner('O modelo está analisando os dados...'):
        try:
            # URL da API. Altere se necessário.
            # Se usar Docker Compose, use o nome do serviço (ex: http://api:8000/predict)
            api_url = "http://127.0.0.1:8000/predict"

            response = requests.post(api_url, data=json.dumps(features))

            if response.status_code == 200:
                prediction_data = response.json()
                diagnostico = prediction_data['diagnostico']
                probabilidade = prediction_data['probabilidade_melanoma']

                # --- Exibir Resultados de forma visual ---
                st.subheader("Resultado da Análise do Modelo")

                if diagnostico == "Melanoma":
                    st.warning(f"**Diagnóstico Sugerido:** {diagnostico}")
                    st.markdown(f"O modelo identificou uma probabilidade de **{probabilidade:.2%}** de a lesão ser um Melanoma.")
                else:
                    st.success(f"**Diagnóstico Sugerido:** {diagnostico}")
                    st.markdown(f"A probabilidade de a lesão ser um Melanoma é de **{probabilidade:.2%}**.")

                # Gráfico de rosca para a probabilidade
                prob_data = pd.DataFrame({
                    'Categoria': ['Melanoma', 'Não-Melanoma'],
                    'Probabilidade': [probabilidade, 1 - probabilidade]
                })
                fig = px.pie(prob_data, values='Probabilidade', names='Categoria',
                             hole=0.4, title='Distribuição de Probabilidade',
                             color_discrete_map={'Melanoma':'#ef553b', 'Não-Melanoma':'#636efa'})
                st.plotly_chart(fig, use_container_width=True)

            else:
                st.error(f"Erro ao obter a previsão da API. Status: {response.status_code}")
                st.json(response.json())

        except requests.exceptions.ConnectionError:
            st.error("Falha na conexão com a API. Verifique se o serviço da API (FastAPI/Uvicorn) está em execução.")
        except Exception as e:
            st.error(f"Ocorreu um erro inesperado: {e}")

Writing app/dashboard.py
